In [1]:
using Pkg
Pkg.activate("."); 
Pkg.instantiate()

  Activating environment at `C:\Users\ESchr\workspace\uni\ATIS3\Single_Project\Project.toml`


In [9]:
using JuMP, CPLEX

In [104]:
#Sets
B = []#Bilateral contracts

W = []#Bilateral contracts
T = []#Bilateral contracts
N = [1,2,3,4]#Bilateral contracts
T = []#Bilateral contracts

NL = length(N)

4

In [105]:
# Risk
β = 0.0
α = 0.95

0.95

In [139]:
co_pv = Model(CPLEX.Optimizer)

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: CPLEX

In [140]:
for n in 2:NL
    print(n)
end

234

In [141]:
@variables(co_pv, begin
        z #Objective function value
        PB[B,W] #Power purchased from bilateral contracts
        s[B,W], Bin #Selection of bilateral contracts
        EG[N,T,W] >= 0 #Energy produced by the self-production unit
        EP[T,W] #Energy traded in the pool
        
        η[W] >= 0 #Auxiliary variable used to calculate the CVaR
        ζ #Auxiliary variable used to calculate the CVaR
    end)

In [142]:
@constraints(co_pv, begin
        LimitContractMin[b in B, w in W], PB[B,W] == PBmin[B]*s[B,W]
        LimitContractMax[b in B, w in W], PB[B,W] <= PBmax[B]*s[B,W]
        
        #attetion hard coded
        SelfProduction1[nl in 1:1, t in T, w in W], EG[nl,t,w] <= EGT[nl]
        SelfProduction[nl in 2:NL, t in T, w in W], EG[nl,t,w] <= EGT[nl] - EGT[nl-1]
        EnergyBalance[t in T, w in W], sum(EG[n,t,w] for n in N) + EP[t,w] + sum(PB[b,w]*d[t] for b in B)#=$Bt(B,T)=# == ED[t]-EPC[t]
        Arbitrage[t in T, w in W], sum(EG[n,t,w] for n in N) >= -EP[t,w]
        #Non_Anticipativity $(KB(B,K) and (A(W,K)=1) and (ord(W) lt card(W))
        #Non_Anticipativity[b in B, w in W, k in K], PB[b,w] == PB[b, w+1]
        CVaRconstr[w in W], sum(sum( λ_B[b,t,w]*PB[b,w]*d[t] for b in B #=B$Bt(B,T)=#) + λ_P[w,t]*EP[t,w] + sum(SG[n]*EG[n,t,w] for n in N) for t in T)- ζ - η[w] == 0        
        
    end)